In [1]:
import random
from pathlib import Path

import anndata as ad
import igraph as ig
import leidenalg as la
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import scanpy as sc
import seaborn as sns
import scipy as sp
import squidpy as sq
from scipy.sparse import find
from sklearn.metrics import adjusted_rand_score

In [2]:
data_dir = Path("./data")
result_dir = Path("./results")

seed = 42

In [3]:
metadata = pd.read_table(
    data_dir / "samples.tsv", usecols=["directory", "n_clusters"]
).set_index("directory")

In [4]:
def get_anndata(path):
    X = sp.io.mmread(path / "counts.mtx").tocsr()

    observations = pd.read_table(path / "observations.tsv", index_col=0)
    features = pd.read_table(path / "features.tsv", index_col=0)

    coordinates = (
        pd.read_table(path / "coordinates.tsv", index_col=0)
        .loc[observations.index, :]
        .to_numpy()
    )

    adata = ad.AnnData(
        X=X, obs=observations, var=features, obsm={"spatial": coordinates}
    )

    return adata

# Leiden

In [5]:
def search_resolution(
    adata, ncluster, start=1, step=0.1, n_iterations=15, seed=42, **kwargs
):
    # adapted from SpaGCN.search_res (https://github.com/jianhuupenn/SpaGCN)
    res = start
    sc.tl.leiden(adata, resolution=res, random_state=seed, **kwargs)
    old_ncluster = adata.obs["leiden"].cat.categories.size
    iter = 1
    while old_ncluster != ncluster:
        old_sign = 1 if (old_ncluster < ncluster) else -1
        sc.tl.leiden(
            adata, resolution=res + step * old_sign, random_state=seed, **kwargs
        )
        new_ncluster = adata.obs["leiden"].cat.categories.size
        if new_ncluster == ncluster:
            res = res + step * old_sign
            # print(f"Recommended res = {res:.2f}")
            return res
        new_sign = 1 if (new_ncluster < ncluster) else -1
        if new_sign == old_sign:
            res = res + step * old_sign
            # print(f"Res changed to {res:.2f}")
            old_ncluster = new_ncluster
        else:
            step = step / 2
            # print(f"Step changed to {step:.2f}")
        if iter > n_iterations:
            # print("Exact resolution not found")
            # print(f"Recommended res =  {res:.2f}")
            return res
        iter += 1
    # print(f"Recommended res = {res:.2f}")
    return res

In [6]:
def preprocess_adata(adata, seed=42, genes=1000, n_pcs=30):
    sc.pp.filter_genes(adata, min_cells=3)
    sc.pp.highly_variable_genes(adata, flavor="seurat_v3", n_top_genes=genes)
    sc.pp.normalize_total(adata, target_sum=1e4)
    sc.pp.log1p(adata)
    sc.tl.pca(adata, n_comps=n_pcs, random_state=seed)
    sc.pp.neighbors(adata, random_state=seed)


def run_leiden(adata, n_clusters, seed, **kwargs):
    res = search_resolution(adata, n_clusters, seed=seed, **kwargs)

    df = adata.obs[["leiden"]]
    df.columns = ["label"]
    return df, res

## Multiplex

In [7]:
def _build_igraph(adjacency, directed=True):
    # adapted from scanpy
    sources, targets, weights = find(adjacency)
    g = ig.Graph(directed=directed)
    g.add_vertices(adjacency.shape[0])
    g.add_edges(list(zip(sources, targets)))
    g.es["weight"] = weights
    if g.vcount() != adjacency.shape[0]:
        raise RuntimeError(
            f"The constructed graph has only {g.vcount()} nodes. "
            "Your adjacency matrix contained redundant nodes."
        )
    return g

In [8]:
def search_spatial_resolution_multiplex(
    adata, ncluster, start=0.4, step=0.1, n_iterations=15, seed=42, **kwargs
):
    # adapted from SpaGCN.search_res (https://github.com/jianhuupenn/SpaGCN)
    res = start
    spatial_partition_kwargs = kwargs.pop("spatial_partition_kwargs", dict())
    spatial_partition_kwargs["resolution_parameter"] = res
    leiden_multiplex(
        adata, spatial_partition_kwargs=spatial_partition_kwargs, seed=seed, **kwargs
    )
    old_ncluster = adata.obs["leiden_multiplex"].cat.categories.size
    iter = 1
    while old_ncluster != ncluster:
        old_sign = 1 if (old_ncluster < ncluster) else -1
        spatial_partition_kwargs["resolution_parameter"] = res + step * old_sign
        leiden_multiplex(
            adata,
            spatial_partition_kwargs=spatial_partition_kwargs,
            seed=seed,
            **kwargs,
        )
        new_ncluster = adata.obs["leiden_multiplex"].cat.categories.size
        if new_ncluster == ncluster:
            res = res + step * old_sign
            # print(f"Recommended res = {res:.2f}")
            return res
        new_sign = 1 if (new_ncluster < ncluster) else -1
        if new_sign == old_sign:
            res = res + step * old_sign
            # print(f"Res changed to {res:.2f}")
            old_ncluster = new_ncluster
        else:
            step = step / 2
            # print(f"Step changed to {step:.2f}")
        if iter > n_iterations:
            # print("Exact resolution not found")
            # print(f"Recommended res = {res:.2f}")
            return res
        iter += 1
    # print(f"Recommended res = {res:.2f}")
    return res

In [9]:
def leiden_multiplex(
    adata,
    key_added: str = "leiden_multiplex",
    directed: tuple[bool, bool] = (True, True),
    use_weights: bool = True,
    n_iterations: int = -1,
    partition_type=la.RBConfigurationVertexPartition,
    scale_graph_weights: tuple[bool, bool] = (False, False),
    layer_weights: tuple[int, int] = (1, 1),
    spatial_partition_kwargs=None,
    latent_partition_kwargs=None,
    diff_threshold: float = 1e-05,
    seed=42,
):
    spatial_distance_key = "spatial_distances"
    latent_distance_key = "connectivities"

    latent_distances = adata.obsp[latent_distance_key]
    spatial_distances = adata.obsp[spatial_distance_key]

    if scale_graph_weights[0]:
        percentile = np.percentile(latent_distances.data, 95)
        latent_distances = latent_distances.multiply(1 / percentile)
    if scale_graph_weights[1]:
        percentile = np.percentile(spatial_distances.data, 95)
        spatial_distances = spatial_distances.multiply(1 / percentile)

    adjacency_latent = _build_igraph(latent_distances, directed=directed[0])
    adjacency_spatial = _build_igraph(spatial_distances, directed=directed[1])

    # parameterise the partitions
    if spatial_partition_kwargs is None:
        spatial_partition_kwargs = dict()
    if latent_partition_kwargs is None:
        latent_partition_kwargs = dict()

    if use_weights:
        spatial_partition_kwargs["weights"] = "weight"
        latent_partition_kwargs["weights"] = "weight"

    latent_part = partition_type(adjacency_latent, **latent_partition_kwargs)
    spatial_part = partition_type(adjacency_spatial, **spatial_partition_kwargs)
    optimiser = la.Optimiser()
    optimiser.set_rng_seed(seed)

    diff = optimiser.optimise_partition_multiplex(
        [latent_part, spatial_part],
        layer_weights=list(layer_weights),
        n_iterations=n_iterations,
    )

    adata.obs[key_added] = np.array(latent_part.membership)
    adata.obs[key_added] = adata.obs[key_added].astype("category")


def run_leiden_multiplex(adata, n_clusters, seed, **kwargs):
    res = search_spatial_resolution_multiplex(adata, n_clusters, seed=seed, **kwargs)

    df = adata.obs[["leiden_multiplex"]]
    df.columns = ["label"]
    return df, res

# MULTISPATI-PCA

In [10]:
import warnings
from typing import TypeAlias, TypeVar

import numpy as np
from numpy.typing import NDArray
from scipy import linalg
from scipy.sparse import csc_array, csc_matrix, csr_array, csr_matrix, issparse
from scipy.sparse import linalg as sparse_linalg
from scipy.sparse import sparray, spmatrix
from sklearn.preprocessing import scale

T = TypeVar("T", bound=np.number)
U = TypeVar("U", bound=np.number)


_Csr: TypeAlias = csr_array | csr_matrix
_Csc: TypeAlias = csc_array | csc_matrix
_X: TypeAlias = np.ndarray | _Csr | _Csc


class MultispatiPCA:
    """
    MULTISPATI-PCA (Principal Component Analysis).
    """

    # TODO, should scaling be part of multispati

    def __init__(
        self,
        n_components: int | tuple[int, int] | None = None,
        *,
        connectivity: sparray | spmatrix,
    ):
        """
        Parameters
        ----------

        n_components : int or tuple[int, int], optional
            Number of components to keep.
            If None, will keep all components (only supported for non-sparse X).
            If an int, it will keep the top `n_components`.
            If a tuple, it will keep the top and bottom `n_components` respectively.
        connectivity : scipy.sparse.sparray or scipy.sparse.spmatrix
            Matrix of row-wise neighbor definitions i.e. c_ij is the connectivity of
            i -> j. Does not have to be symmetric. Can be a binary adjacency matrix
            or a matrix of connectivities in which case c_ij should be larger
            if i and j are close.
            A distance matrix should transformed to connectivities by e.g.
            calculating :math:`1-d/d_{max}` beforehand.
        Raises
        ------
        ValueError
            If connectivity is not a square matrix.
        ZeroDivisionError
            If one of the observations has no neighbors.
        """
        self._fitted = False
        W = csr_array(connectivity)
        if W.shape[0] != W.shape[1]:
            raise ValueError("`connectivity` must be square")
        self.W = self._normalize_connectivities(W)

        n = self.W.shape[0]

        self._n_neg = 0
        if n_components is None:
            self._n_pos = n_components
        else:
            if isinstance(n_components, int):
                if n_components > n:
                    warnings.warn(
                        "`n_components` should be less or equal than "
                        f"#rows of `connectivity`. Using {n} components."
                    )
                self._n_pos = min(n_components, n)
            elif isinstance(n_components, tuple) and len(n_components) == 2:
                if n < n_components[0] + n_components[1]:
                    warnings.warn(
                        "Sum of `n_components` should be less or equal than "
                        f"#rows of `connectivity`. Using {n} components."
                    )
                    self._n_pos = n
                else:
                    self._n_pos, self._n_neg = n_components
            else:
                raise ValueError("`n_components` must be None, int or (int, int)")

    @staticmethod
    def _normalize_connectivities(W: csr_array) -> csr_array:
        # normalize rowsums to 1 for better interpretability
        # TODO can't handle points without neighbors because of division by zero
        return W.multiply(1 / W.sum(axis=1)[:, np.newaxis])

    def fit(self, X: _X):
        """
        Fit MULTISPATI-PCA projection.

        Parameters
        ----------
        X : numpy.ndarray or scipy.sparse.csr_array or scipy.sparse.csc_array
            Array of observations x features.

        Raises
        ------
        ValueError
            If `X` has not the same number of rows like `connectivity`.
            If `n_components` is None and `X` is sparse.
            If (sum of) `n_components` is larger than the smaller dimension of `X`.
        """
        if issparse(X):
            X = csc_array(X)

        assert isinstance(X, (np.ndarray, csc_array))
        if X.shape[0] != self.W.shape[0]:
            raise ValueError("#rows in `X` must be same as size of `connectivity`")
        if self._n_pos is None:
            if issparse(X):
                raise ValueError(
                    "`n_components` is None, but `X` is a sparse matrix. None is only "
                    "supported for dense matrices."
                )
        elif (self._n_pos + self._n_neg) > X.shape[1]:
            n, d = X.shape
            n_comp = self._n_pos + self._n_neg
            n_comp_max = min(n, d)
            raise ValueError(
                f"Requested {n_comp} components but given `X` at most {n_comp_max} "
                "can be calculated."
            )

        # Data must be scaled, avoid mean-centering for sparse
        X = scale(X, with_mean=not issparse(X))

        eig_val, eig_vec = self._multispati_eigendecomposition(X, self.W)

        self.components_ = eig_vec
        self.eigenvalues_ = eig_val
        self._fitted = True

    def _multispati_eigendecomposition(
        self, X: _X, W: _Csr
    ) -> tuple[NDArray[np.float64], NDArray[np.float64]]:
        # X: beads/bin x gene, must be standardized
        # W: row-wise definition of neighbors, row-sums should be 1
        def remove_zero_eigenvalues(
            eigen_values: NDArray[T], eigen_vectors: NDArray[U], n: int
        ) -> tuple[NDArray[T], NDArray[U]]:
            keep_idx = np.sort(np.argpartition(np.abs(eigen_values), -n)[-n:])

            return eigen_values[keep_idx], eigen_vectors[:, keep_idx]

        n, d = X.shape

        H = (X.T @ (W + W.T) @ X) / (2 * n)
        # TODO handle sparse based on density?
        if issparse(H):
            # TODO fix can't return all eigenvalues of sparse matrix
            # TODO check that number of eigenvalues does not exceed d
            if self._n_pos is None:
                raise ValueError
            elif self._n_pos == 0:
                eig_val, eig_vec = sparse_linalg.eigsh(H, k=self._n_neg, which="SA")
            elif self._n_neg == 0:
                eig_val, eig_vec = sparse_linalg.eigsh(H, k=self._n_pos, which="LA")
            else:
                n_comp = 2 * max(self._n_neg, self._n_pos)
                eig_val, eig_vec = sparse_linalg.eigsh(H, k=n_comp, which="BE")
                component_indices = self._get_component_indices(
                    n_comp, self._n_pos, self._n_neg
                )
                eig_val = eig_val[component_indices]
                eig_vec = eig_vec[:, component_indices]

        else:
            if self._n_pos is None:
                eig_val, eig_vec = linalg.eigh(H)
                if n < d:
                    eig_val, eig_vec = remove_zero_eigenvalues(eig_val, eig_vec, n)
            elif self._n_pos == 0:
                eig_val, eig_vec = linalg.eigh(H, subset_by_index=[0, self._n_neg])
            elif self._n_neg == 0:
                eig_val, eig_vec = linalg.eigh(
                    H, subset_by_index=[d - self._n_pos, d - 1]
                )
            else:
                eig_val, eig_vec = linalg.eigh(H)
                component_indices = self._get_component_indices(
                    d, self._n_pos, self._n_neg
                )
                eig_val = eig_val[component_indices]
                eig_vec = eig_vec[:, component_indices]

        return np.flip(eig_val), np.fliplr(eig_vec)

    @staticmethod
    def _get_component_indices(n: int, n_pos: int, n_neg: int) -> list[int]:
        if n_pos + n_neg > n:
            return list(range(n))
        else:
            return list(range(n_neg)) + list(range(n - n_pos, n))

    def transform(self, X: _X) -> np.ndarray:
        """
        Transform the data using fitted MULTISPATI-PCA projection.

        Parameters
        ----------
        X : numpy.ndarray or scipy.sparse.csr_array or scipy.sparse.csc_array
            Array of observations x features.

        Returns
        -------
        numpy.ndarray

        Raises
        ------
        ValueError
            If instance has not been fitted.
        """
        # Data must be scaled, avoid mean-centering for sparse
        if not self._fitted:
            self._not_fitted()

        X = scale(X, with_mean=not issparse(X))
        return X @ self.components_

    def fit_transform(self, X: _X) -> np.ndarray:
        """
        Fit the MULTISPATI-PCA projection and transform the data.

        Parameters
        ----------
        X : numpy.ndarray or scipy.sparse.csr_array or scipy.sparse.csc_array
            Array of observations x features.

        Returns
        -------
        numpy.ndarray
        """
        self.fit(X)
        return self.transform(X)

    def transform_spatial_lag(self, X: _X) -> np.ndarray:
        """
        Transform the data using fitted MULTISPATI-PCA projection and calculate the
        spatial lag.

        Parameters
        ----------
        X : numpy.ndarray or scipy.sparse.csr_array or scipy.sparse.csc_array
            Array of observations x features.

        Returns
        -------
        numpy.ndarray

        Raises
        ------
        ValueError
            If instance has not been fitted.
        """
        if not self._fitted:
            self._not_fitted()
        return self.W @ self.transform(X)

    def variance_moranI_decomposition(self, X: _X) -> tuple[np.ndarray, np.ndarray]:
        """
        Calculate the decomposition of the variance and Moran's I for `n_components`.

        Parameters
        ----------
        X : numpy.ndarray or scipy.sparse.csr_array or scipy.sparse.csc_array
            Array of observations x features.

        Returns
        -------
        tuple[numpy.ndarray, numpy.ndarray]
            Variance and Moran's I.

        Raises
        ------
        ValueError
            If instance has not been fitted.
        """
        if not self._fitted:
            self._not_fitted()
        transformed = self.transform(X)
        lag = self.transform_spatial_lag(X)

        # vector of row_Weights from dudi.PCA
        # (we only use default row_weights i.e. 1/n anyways)
        w = 1 / X.shape[0]

        variance = np.sum(transformed * transformed * w, axis=0)
        moran = np.sum(transformed * lag * w, axis=0) / variance

        return variance, moran

    def moransI_bounds(
        self, *, sparse_approx: bool = True
    ) -> tuple[float, float, float]:
        """
        Calculate the minimum and maximum bound for Moran's I given the `connectivity`
        and the expected value given the #observations.

        Parameters
        ----------
        sparse_approx : bool
            Only applicable if `connectivity` is sparse.
        Returns
        -------
        tuple[float, float, float]
            Minimum bound, maximum bound, and expected value.
        """

        # following R package adespatial::moran.bounds
        # sparse approx is following adegenet sPCA as shown in screeplot/summary
        def double_center(W):
            if issparse(W):
                W = W.toarray()

            row_means = np.mean(W, axis=1, keepdims=True)
            col_means = np.mean(W, axis=0, keepdims=True) - np.mean(row_means)

            return W - row_means - col_means

        # ensure symmetry
        W = 0.5 * (self.W + self.W.T)

        n_sample = W.shape[0]
        s = n_sample / np.sum(W)  # 1 if original W has rowSums or colSums of 1

        if not issparse(W) or not sparse_approx:
            W = double_center(W)

        if issparse(W):
            eigen_values = s * sparse_linalg.eigsh(
                W, k=2, which="BE", return_eigenvectors=False
            )
        else:
            eigen_values = s * linalg.eigvalsh(W, overwrite_a=True)

        I_0 = -1 / (n_sample - 1)
        I_min = min(eigen_values)
        I_max = max(eigen_values)

        return I_min, I_max, I_0

    def _not_fitted(self):
        raise ValueError(
            "This MultispatiPCA instance is not fitted yet. "
            "Call 'fit' with appropriate arguments first."
        )

# Results

## Impact of layer weight ratio

In [11]:
sample = metadata.loc["Br8100_151673", :]

In [12]:
n_genes = 3_000
n_pcs = 30

out_dir = result_dir / "weightratio_impact" / sample.name

adata = get_anndata(data_dir / sample.name)
preprocess_adata(adata, genes=n_genes, n_pcs=n_pcs, seed=seed)

leiden_df, res = run_leiden(adata, sample.n_clusters, seed=seed)

out_dir.mkdir(parents=True, exist_ok=True)
leiden_df.to_csv(out_dir / "leiden.tsv", sep="\t", index_label="")

sq.gr.spatial_neighbors(adata, coord_type="grid", n_neighs=6)

for weight_ratio in [0, 0.2, 0.4, 0.6, 0.8, 1, 5, 10]:
    multiplex_df, res_multi = run_leiden_multiplex(
        adata,
        sample.n_clusters,
        directed=(False, False),
        scale_graph_weights=(False, False),
        layer_weights=(1, weight_ratio),
        latent_partition_kwargs={"resolution_parameter": res},
        seed=seed,
    )
    multiplex_df.to_csv(
        out_dir / f"spatial_leiden_w{weight_ratio:.1f}.tsv", sep="\t", index_label=""
    )

/home/jovyan/conda_envs/scverse_3_10/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
/tmp/ipykernel_61478/3929316464.py:6: FutureWarning: In the future, the default backend for leiden will be igraph instead of leidenalg.

 To achieve the future defaults please pass: flavor="igraph" and n_iterations=2.  directed must also be False to work with igraph's implementation.
  sc.tl.leiden(adata, resolution=res, random_state=seed, **kwargs)


## Cluster all samples

### HVGs

In [13]:
n_pcs = 30
n_genes = 3_000
weight_spatial = 0.8


for name, sample in metadata.iterrows():
    print("Processing " + name)

    sample_dir = data_dir / name
    out_dir = result_dir / name

    adata = get_anndata(sample_dir)
    preprocess_adata(adata, genes=n_genes, n_pcs=n_pcs, seed=seed)

    label_leiden, _ = run_leiden(adata, sample.n_clusters, seed=seed)

    # Multiplex
    sq.gr.spatial_neighbors(adata, coord_type="grid", n_neighs=6)
    label_leiden_multi, _ = run_leiden_multiplex(
        adata,
        sample.n_clusters,
        directed=(False, False),
        scale_graph_weights=(False, False),
        layer_weights=(1, weight_spatial),
        latent_partition_kwargs={"resolution_parameter": res},
        seed=seed,
    )

    ## Write output
    out_dir.mkdir(parents=True, exist_ok=True)
    label_leiden.to_csv(out_dir / "leiden.tsv", sep="\t", index_label="")
    label_leiden_multi.to_csv(out_dir / "spatial_leiden.tsv", sep="\t", index_label="")

Processing Br5292_151507
Processing Br5292_151508
Processing Br5292_151509
Processing Br5292_151510
Processing Br5595_151669
Processing Br5595_151670
Processing Br5595_151671
Processing Br5595_151672
Processing Br8100_151673
Processing Br8100_151674
Processing Br8100_151675
Processing Br8100_151676


### SVGs

In [14]:
n_pcs = 30
n_genes = 3_000
weight_spatial = 0.8

for name, sample in metadata.iterrows():
    print("Processing " + name)

    sample_dir = data_dir / name
    out_dir = result_dir / name

    adata = get_anndata(sample_dir)
    preprocess_adata(adata, genes=n_genes, n_pcs=n_pcs, seed=seed)
    sq.gr.spatial_neighbors(adata, coord_type="grid", n_neighs=6)
    sq.gr.spatial_autocorr(adata, genes=adata.var_names, mode="moran", seed=seed)
    genes = adata.uns["moranI"].nlargest(n_genes, columns="I", keep="all").index
    adata.obsm["X_svg_pca"] = sc.tl.pca(
        adata[:, genes].X, n_comps=n_pcs, random_state=seed
    )
    sc.pp.neighbors(adata, use_rep="X_svg_pca", random_state=seed)

    label_leiden, _ = run_leiden(adata, sample.n_clusters, seed=seed)

    # Multiplex
    sq.gr.spatial_neighbors(adata, coord_type="grid", n_neighs=6)
    label_leiden_multi, _ = run_leiden_multiplex(
        adata,
        sample.n_clusters,
        directed=(False, False),
        scale_graph_weights=(False, False),
        layer_weights=(1, weight_spatial),
        latent_partition_kwargs={"resolution_parameter": res},
        seed=seed,
    )

    # Multiplex and MULTISPATI-PCA
    adata.obsm["X_mspca"] = MultispatiPCA(
        30, connectivity=adata.obsp["connectivities"]
    ).fit_transform(adata[:, genes].X)
    sc.pp.neighbors(adata, use_rep="X_mspca", random_state=seed)

    label_leiden_msPCA, res = run_leiden(adata, sample.n_clusters, seed=seed)
    label_leiden_multi_msPCA, _ = run_leiden_multiplex(
        adata,
        sample.n_clusters,
        directed=(False, False),
        scale_graph_weights=(False, False),
        layer_weights=(1, weight_spatial),
        latent_partition_kwargs={"resolution_parameter": res},
        seed=seed,
    )

    ## Write output
    out_dir.mkdir(parents=True, exist_ok=True)
    label_leiden.to_csv(out_dir / "leiden_svg.tsv", sep="\t", index_label="")
    label_leiden_multi.to_csv(
        out_dir / "spatial_leiden_svg.tsv", sep="\t", index_label=""
    )
    label_leiden_msPCA.to_csv(
        out_dir / "leiden_svg_multispati.tsv", sep="\t", index_label=""
    )
    label_leiden_multi_msPCA.to_csv(
        out_dir / "spatial_leiden_svg_multispati.tsv", sep="\t", index_label=""
    )

Processing Br5292_151507
Processing Br5292_151508
Processing Br5292_151509
Processing Br5292_151510
Processing Br5595_151669
Processing Br5595_151670
Processing Br5595_151671
Processing Br5595_151672
Processing Br8100_151673
Processing Br8100_151674
Processing Br8100_151675
Processing Br8100_151676
